## External code for model set up
This notebook will hold code used in the initial set up of the model but is not needed for every run (ie transforming the large scale 10 meter dem from epsg 4326 to epsg 32610 only needs to be done once)

In [ ]:
spath = "C://Users/ajcalder/Box/Research_Calderwood/dem"

# 1 meter dem
raster_name = spath+"/model_dem.tif"

rio = Raster.load(raster_name)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect='equal')

ax = rio.plot(ax=ax)
plt.colorbar(ax.images[0], shrink=0.7);

In [ ]:
# 10 meter dem
raster_name = spath+"/USGS_ten_meter_dem/USGS_13_n39w122_10meterdem.tif"

rio10 = Raster.load(raster_name)

In [ ]:
# Convert 10 meter dem crs from lat long to utm zone 10n
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:32610'

raster_name = spath+"/USGS_ten_meter_dem/USGS_13_n39w122_10meterdem.tif"
with rasterio.open(raster_name) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(spath+'/USGS_ten_meter_dem/transformed.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

# Model vertices, shapefile

In [ ]:
# Get vertexes of model domain
ll = mg.get_coords(0, 0) #lower left
lr = mg.get_coords(nrow*delr, 0) #lower right
ur = mg.get_coords(nrow*delr, ncol*delc) #upper right
ul = mg.get_coords(0, ncol*delc) #upper left
print(ll, lr, ur, ul)

# Shapefile of model bounds
vertices = np.stack(np.asarray((ll,lr, ur, ul)))
vertices

In [ ]:
geoms = Polygon(vertices)
geoms.plot() # this feature requires descartes
geoms.type

# Saving a polygon to a shapefile

In [ ]:
# How to save a polygon to shapefile
import shapely
w = shapefile.Writer('polygon')
w.field('name', 'C')
w.poly([vertices])
w.record('polygon1')
w.close()

# Raster cropping

In [ ]:
t0 = time.time()
rio10_utm.crop(vertices, invert=False)
crop_time = time.time() - t0

In [ ]:
rio10_utm.plot()

In [ ]:
rio10_utm.write()